In [ ]:
import os
import time
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torchvision.models import alexnet

In [ ]:
model = alexnet()
convs = []
for i,k in enumerate(model.modules()):
    if isinstance(k, nn.Conv2d):
        convs.append(k)

In [ ]:
l1 = convs[0].weight

In [ ]:
for j in range(l1.shape[1]): # itera sobre cada canal
    X = None
    y = None
    for i,w in enumerate(l1): #itera sobre cada filtro
        if X == None:
            X = torch.reshape(w[j], [1,w[j].shape[1]**2])  # <----- Problema: repetindo 1o elemento
        else:
            y = torch.reshape(w[0], [1,w[j].shape[1]**2])
            X = torch.cat((X,y))
    dist = _mahalanobis(X)

In [ ]:
x = X[0] #vector1
y = X[2] #vector 2
### mahalanobis params
VI = torch.inverse(cov(X)) #inverse of covariance matrix
p2 = (x - y).unsqueeze(0)
p1 = (x-y).unsqueeze(0).t()

In [ ]:
p1.shape

In [ ]:
p2.shape

In [ ]:
torch.sqrt(torch.mm(p2,torch.mm(VI,p1))) # this?

In [ ]:
torch.sqrt(torch.mm(torch.mm(p2,VI),p1)) # or this?

In [ ]:
_mahalanobis(X)

In [ ]:
def _mahalanobis(X):
    VI = torch.inverse(cov(X)) #covariance matrix
    total_dist = 0
    for i,v in enumerate(X):
        dist = 0
        for j,u in enumerate(X):
            if i == j:
                continue
            x = (v-u).unsqueeze(0).t()
            y = (v - u).unsqueeze(0)
            
            dist = torch.sqrt(torch.mm(torch.mm(y,VI),x))
            
            total_dist +=dist
            print(dist)
    return total_dist

In [ ]:
range(l1.shape[0])

In [ ]:
S = torch.inverse(cov(w1))
mu = torch.mean(w1)

In [ ]:
import torch

def cov(m, rowvar=False):
    '''Estimate a covariance matrix given data.

    Covariance indicates the level to which two variables vary together.
    If we examine N-dimensional samples, `X = [x_1, x_2, ... x_N]^T`,
    then the covariance matrix element `C_{ij}` is the covariance of
    `x_i` and `x_j`. The element `C_{ii}` is the variance of `x_i`.

    Args:
        m: A 1-D or 2-D array containing multiple variables and observations.
            Each row of `m` represents a variable, and each column a single
            observation of all those variables.
        rowvar: If `rowvar` is True, then each row represents a
            variable, with observations in the columns. Otherwise, the
            relationship is transposed: each column represents a variable,
            while the rows contain observations.

    Returns:
        The covariance matrix of the variables.
    '''
    if m.dim() > 2:
        raise ValueError('m has more than 2 dimensions')
    if m.dim() < 2:
        m = m.view(1, -1)
    if not rowvar and m.size(0) != 1:
        m = m.t()
    # m = m.type(torch.double)  # uncomment this line if desired
    fact = 1.0 / (m.size(1) - 1)
    m -= torch.mean(m, dim=1, keepdim=True)
    mt = m.t()  # if complex: mt = m.t().conj()
    return fact * m.matmul(mt).squeeze()